<a href="https://colab.research.google.com/github/shiehn/elixirs/blob/main/elixir_demucs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Check if dawnet-client is installed, if not then install it
try:
    import dawnet_client
except ImportError:
    !pip install dawnet-client
    import dawnet_client

# Check if demucs is installed, if not then install it
try:
    import demucs
except ImportError:
    !pip install demucs
    import demucs

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

import os
import subprocess
import stat
import time

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "0b03beea-6d87-4c73-9fa5-0616aa6f01dd" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(input_file: DAWNetFilePath):
    try:
        command = ['python3', '-m', 'demucs']

        print(f"input_file: {input_file}")

        # Add the audio file path
        command.append(input_file)

        # Execute the command
        subprocess.run(command)

        model = 'htdemucs'
        input_file_without_extension = os.path.splitext(os.path.basename(input_file))[0]
        path = os.path.join('separated', model, input_file_without_extension)

        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)

            await dawnet.output().add_file(full_path)

        await dawnet.output().add_message("Stem Seperation Complete")

        return True
    except Exception as e:

        await dawnet.output().add_error(f"Method encountered an error: {e}")
        return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("Stem Separation - DEMUCS")
dawnet.set_description("Send an audio file & have DAWNet split stems and return them")
dawnet.register_method(dawnet_func)

dawnet.set_output_target_format('wav')
dawnet.set_output_target_channels(2)
dawnet.set_output_target_sample_rate(44100)
dawnet.set_output_target_bit_depth(16)

dawnet.connect_to_server()